In [2]:
import os
import joblib
import numpy as np
import pandas as pd
import polars as pl

import pandas.api.types
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import lifelines
from lifelines.utils import concordance_index

import sys

print(sys.version)
for i in [np, pd, pl, mpl, sns, lifelines, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        print(i.__name__)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
numpy 1.26.4
pandas 2.2.3
polars 1.9.0
matplotlib 3.7.5
seaborn 0.12.2
lifelines 0.30.0
sklearn 1.2.2
lightgbm 4.5.0
xgboost 2.0.3
catboost 1.2.7


In [3]:
import dproc, sgutil, sgpp, sgml, custpp
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit, train_test_split
from sklearn.impute import SimpleImputer
from lifelines import NelsonAalenFitter

In [9]:
data_path = 'data'
img_path = 'img'
result_path = 'result'
model_path = 'model'

sc = sgutil.SGCache(img_path, result_path)

X_4 = [
    'psych_disturb', 'diabetes', 'arrhythmia', 'renal_issue', 'pulm_severe', 'obesity', 'hepatic_severe', 
    'prior_tumor', 'peptic_ulcer','rheum_issue', 'hepatic_mild', 'cardiac','pulm_moderate'
]

X_int_fm = [
    'comorbidity_score', 'hla_high_res_10', 'hla_high_res_6', 'hla_high_res_8', 'hla_low_res_10',
    'hla_low_res_6', 'hla_low_res_8', 'hla_match_a_high', 'hla_match_a_low', 'hla_match_b_high',
    'hla_match_b_low', 'hla_match_drb1_low', 'hla_match_c_high', 'hla_match_c_low', 'hla_match_dqb1_high', 'hla_match_dqb1_low',
    'hla_match_drb1_high', 'hla_nmdp_6', 'karnofsky_score'
]

X_2 = [
    ('graft_type', {'peripheral blood': 0, 'bone marrow': 1}),
    ('prod_type', {'pb': 0, 'bm': 1}),
    ('vent_hist', {'no': -1, 'yes': 1}),
    ('rituximab', {'no': -1, 'yes': 1}),
    ('mrd_hct', {'negative': -1, 'positive': 1}),
    ('in_vivo_tcd', {'no': -1, 'yes': 1}),
    ('melphalan_dose', {'n/a, mel not given': -1, 'mel': 1})
]
X_na = X_4 +  X_int_fm +  ['donor_age']
X_nom_na = ['cmv_status', 'conditioning_intensity', 'cyto_score', 'cyto_score_detail', 'donor_related',
    'dri_score', 'ethnicity', 'gvhd_proph', 'sex_match', 'tce_div_match', 'tce_imm_match', 'tce_match'
]
X_nom_nna = ['prim_disease_hct', 'race_group', 'tbi_status']
p2 = make_pipeline(
    sgpp.PolarsProcessor({'ID': pl.Int64}),
    sgpp.PandasCoverter(index_col = 'ID'),
    custpp.CIBMTTransformer(X_2, X_4, X_na, X_nom_na + X_nom_nna),
    sgpp.ApplyWrapper(SimpleImputer(strategy='most_frequent').set_output(transform='pandas'), X_int_fm, postfix = '_fm'),
    sgpp.ApplyWrapper(SimpleImputer(strategy='mean').set_output(transform='pandas'), ['donor_age'], postfix = '_fm'),
    sgpp.ApplyWrapper(sgpp.CatArrangerFreq(1, 'na', 'na'), X_nom_na),
    sgpp.ApplyWrapper(sgpp.CatOOVFilter(), X_nom_nna),
)
df_train = p2.fit_transform(['data/train.csv'])
joblib.dump(p2, os.path.join('model', 'p2.joblib'))

['model/p2.joblib']

In [10]:
def get_naf(df, time_col='efs_time', event_col='efs'):
    naf = NelsonAalenFitter()
    naf.fit(durations=df['efs_time'], event_observed=df['efs'])
    return naf.cumulative_hazard_at_times(df['efs_time']).values * -1
df_train['naf'] = get_naf(df_train, time_col='efs_time', event_col='efs')

In [16]:
X_bool = ['graft_type', 'prod_type']
X_tri = [
    'arrhythmia', 'cardiac', 'diabetes', 'hepatic_mild', 'hepatic_severe',
    'in_vivo_tcd', 'melphalan_dose', 'mrd_hct', 'obesity', 'peptic_ulcer',
    'prior_tumor', 'psych_disturb', 'pulm_moderate', 'pulm_severe', 'renal_issue',
    'rheum_issue', 'rituximab', 'vent_hist'
]
X_nom = [
    'cmv_status', 'conditioning_intensity', 'cyto_score', 'cyto_score_detail', 'donor_related',
    'dri_score', 'ethnicity', 'gvhd_proph', 'prim_disease_hct', 'race_group', 'sex_match',
    'tbi_status', 'tce_div_match', 'tce_imm_match', 'tce_match'
]
X_na = [
    'arrhythmia_na', 'cardiac_na', 'diabetes_na', 'hepatic_mild_na', 'hepatic_severe_na',
    'obesity_na', 'peptic_ulcer_na', 'prior_tumor_na', 'psych_disturb_na', 'pulm_moderate_na',
    'pulm_severe_na', 'renal_issue_na', 'rheum_issue_na'
]
X_cont = ['age_at_hct', 'donor_age']
X_int = [
    'comorbidity_score', 'hla_high_res_10', 'hla_high_res_6', 'hla_high_res_8', 'hla_low_res_10',
    'hla_low_res_6', 'hla_low_res_8', 'hla_match_a_high', 'hla_match_a_low', 'hla_match_b_high',
    'hla_match_b_low', 'hla_match_drb1_low', 'hla_match_c_high', 'hla_match_c_low', 'hla_match_dqb1_high', 'hla_match_dqb1_low',
    'hla_match_drb1_high', 'hla_nmdp_6', 'karnofsky_score', 'year_hct'
]


In [17]:
def score(df, prds):
    return df.groupby('race_group', observed=True).apply(
        lambda x: concordance_index(x['efs_time'], -prds.loc[x.index], x['efs']), include_groups=False
    ).pipe(
        lambda x: float(x.mean() - x.std(ddof=0))
    )
def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index),
    'score_func': score,
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': 'naf',
}

kf = KFold(n_splits=10, shuffle=True, random_state = 123)
ss = ShuffleSplit(n_splits=1, random_state = 123)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor)
cb_adapter = sgml.CBAdapter(cb.CatBoostRegressor)

# LGB2

In [18]:
lgb2 = sgml.CVModel('model', 'lgb2', kf, config, lgb_adapter).load_if_exists()

In [37]:
hparams = {
    'model_params': {'num_leaves':  15, 'n_estimators':  3000, 'colsample_bytree': 0.25, 'learning_rate': 0.02},
    'X_num': X_tri + X_cont + X_int + X_na + X_bool, 
    'X_cat': X_nom, 'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1},
    #'validation_fraction': 0.1
}
#result = lgb2.adhoc(df_train, ss, hparams)
result = lgb2.cv(df_train, hparams)

Fold:   0%|          | 0/10 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

...

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

In [38]:
np.mean(result['valid_scores']), np.mean(result['train_scores'])#,result['model_result'][0]['valid_result'].idxmin()

([0.6701505975227752,
  0.6580409512349454,
  0.669256522454466,
  0.6672902579835366,
  0.671588071264131,
  0.6751287789795556,
  0.6762450092215748,
  0.6661822745448618,
  0.6719850450163332,
  0.6597809108573061],
 [0.7664814073102613,
  0.7660909785617896,
  0.7659540801742498,
  0.7655554532339128,
  0.7659232248855976,
  0.7651324943753222,
  0.7659994747628511,
  0.7653109285292482,
  0.7657546979084396,
  0.7656552531118537])

In [40]:
score(df_train, lgb2.cv_best_['prd'].sort_index())

0.6759646058725581